# Experimento: Match bills y BCN

Notebook minimal para probar si podemos vincular registros en la tabla `bills` con el historial de versiones publicado por BCN via SPARQL.


In [3]:
# Imports y setup basico
import os, re, sqlite3
from pathlib import Path
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display

def resolve_db_path():
    env = os.getenv('PARLAMENTO_DB_PATH')
    candidates = [
        env if env else None,
        'data/database/parlamento.db',
        '../data/database/parlamento.db',
        '../../data/database/parlamento.db',
    ]
    candidates = [c for c in candidates if c]
    for c in candidates:
        if os.path.exists(c):
            return str(Path(c).resolve())
    # Fallback: buscar en el arbol del repo
    try:
        for root, dirs, files in os.walk('.'):
            if 'parlamento.db' in files:
                return str(Path(root, 'parlamento.db').resolve())
    except Exception:
        pass
    return str(Path('data/database/parlamento.db').resolve())

DB_PATH = resolve_db_path()
BCN_SPARQL_ENDPOINT = os.getenv('BCN_SPARQL_ENDPOINT', 'https://datos.bcn.cl/sparql')
SAMPLE_SIZE = int(os.getenv('BCN_SAMPLE_SIZE', '5'))

print('cwd =', os.getcwd())
print('DB_PATH =', DB_PATH, '| exists:', os.path.exists(DB_PATH))
print('BCN SPARQL =', BCN_SPARQL_ENDPOINT)


cwd = c:\Users\benja\OneDrive\Desktop\GitHub 2025\prototype-diputados-chatbot\notebooks
DB_PATH = C:\Users\benja\OneDrive\Desktop\GitHub 2025\prototype-diputados-chatbot\data\database\parlamento.db | exists: True
BCN SPARQL = https://datos.bcn.cl/sparql


In [4]:
# Conectar a la base y seleccionar candidatos
conn = None
try:
    db_dir = os.path.dirname(DB_PATH) or '.'
    if not os.path.isdir(db_dir):
        print('Directorio de DB no existe:', db_dir)
    conn = sqlite3.connect(DB_PATH)
    print('Conexion OK a', DB_PATH)
except Exception as e:
    print('Error conectando a DB:', e, '| DB_PATH =', DB_PATH)

df_candidatos = pd.DataFrame()
if conn is not None:
    try:
        q1 = (
            'SELECT bill_id, titulo, ley_numero ' +
            'FROM bills ' +
            'WHERE ley_numero IS NOT NULL ' +
            'ORDER BY ley_fecha_publicacion DESC ' +
            'LIMIT ?;'
        )
        df_candidatos = pd.read_sql_query(q1, conn, params=(SAMPLE_SIZE,))
        if df_candidatos.empty:
            q2 = 'SELECT bill_id, titulo, ley_numero FROM bills LIMIT ?;'
            df_candidatos = pd.read_sql_query(q2, conn, params=(SAMPLE_SIZE,))
        print('Candidatos:', len(df_candidatos))
        display(df_candidatos.head(10))
    except Exception as e:
        print('Error consultando bills:', e)


Conexion OK a C:\Users\benja\OneDrive\Desktop\GitHub 2025\prototype-diputados-chatbot\data\database\parlamento.db
Candidatos: 5


,bill_id,titulo,ley_numero
0,17450-11,"Modifica el DFL N° 1, de Salud, de 2005, para ...",None
1,17377-13,Modifica el Código del Trabajo para permitir p...,None
2,17376-07,Modifica la ley N° 20.084 para autorizar el tr...,None
3,17357-18,Modifica la ley N° 14.908 para perfeccionar lo...,None
4,17356-06,"Modifica la ley N° 19.175, sobre gobierno y ad...",None


In [5]:
# Funciones auxiliares
def limpiar_ley_numero(x):
    if not x: return None
    s = str(x)
    s = re.sub(r'(?i)\bley\b|\bnº\b|\bn°\b|\bnumero\b|\bnúmero\b|\s', '', s)
    s = s.replace('.', '').replace(',', '')
    return s or None

def armar_query(candidates):
    # candidates: lista de strings
    quoted = ' '.join(["'{}'".format(c) for c in candidates])
    parts = [
        'PREFIX bcnnorms: <http://datos.bcn.cl/ontologies/bcn-norms#>',
        'PREFIX dc: <http://purl.org/dc/elements/1.1/>',
        '',
        'SELECT ?normaRaiz ?version ?tituloVersion ?fechaVersion ?esUltimaVersion ?documentoHtml',
        'WHERE {',
        '  VALUES ?ident { ' + quoted + ' }',
        '  ?normaRaiz dc:identifier ?ident .',
        '  ?version bcnnorms:versionOf ?normaRaiz .',
        '  OPTIONAL { ?version dc:title ?tituloVersion . }',
        '  OPTIONAL { ?version bcnnorms:versionDate ?fechaA . }',
        '  OPTIONAL { ?version bcnnorms:validFrom ?fechaB . }',
        '  BIND(COALESCE(?fechaA, ?fechaB) AS ?fechaVersion)',
        '  OPTIONAL { ?version bcnnorms:isLatestVersion ?esA . }',
        '  BIND(COALESCE(?esA) AS ?esUltimaVersion)',
        '  OPTIONAL { ?version bcnnorms:hasHtmlDocument ?htmlA . }',
        '  OPTIONAL { ?version bcnnorms:hasHTMLDocument ?htmlB . }',
        '  OPTIONAL { ?version bcnnorms:htmlDocument ?htmlC . }',
        '  BIND(COALESCE(?htmlA, ?htmlB, ?htmlC) AS ?documentoHtml)',
        '}',
        'ORDER BY DESC(?fechaVersion)',
    ]
    return ''.join(parts)

def fetch_historial(bill_id=None, ley_numero=None):
    cands = []
    ln = limpiar_ley_numero(ley_numero)
    if ln: cands.append(ln)
    if bill_id:
        cands.append(str(bill_id))
        cands.append(re.sub(r'[^0-9A-Za-z]', '', str(bill_id)))
    if not cands:
        return pd.DataFrame()
    q = armar_query(cands)
    s = SPARQLWrapper(BCN_SPARQL_ENDPOINT)
    s.setQuery(q)
    s.setReturnFormat(JSON)
    try:
        s.setTimeout(60)
    except Exception:
        pass
    try:
        res = s.query().convert()
        rows = []
        for r in res.get('results', {}).get('bindings', []):
            rows.append({
                'norma_raiz_uri': r.get('normaRaiz', {}).get('value'),
                'norma_instancia_uri': r.get('version', {}).get('value'),
                'titulo_version': r.get('tituloVersion', {}).get('value'),
                'fecha_version': r.get('fechaVersion', {}).get('value'),
                'es_ultima_version': r.get('esUltimaVersion', {}).get('value'),
                'url_texto_html': r.get('documentoHtml', {}).get('value'),
            })
        return pd.DataFrame(rows)
    except Exception as e:
        print('Error SPARQL:', e)
        return pd.DataFrame()


In [11]:
# Probar con un candidato
df_hist = pd.DataFrame()
if not df_candidatos.empty:
    row = df_candidatos.iloc[7]
    print('Ejemplo bill_id =', row['bill_id'], '| ley_numero =', row.get('ley_numero'))
    df_hist = fetch_historial(row['bill_id'], row.get('ley_numero'))
    display(df_hist.head(20))
else:
    print('Sin candidatos en DB para probar.')


IndexError: single positional indexer is out-of-bounds

In [12]:
df_candidatos

,bill_id,titulo,ley_numero
0,17450-11,"Modifica el DFL N° 1, de Salud, de 2005, para ...",None
1,17377-13,Modifica el Código del Trabajo para permitir p...,None
2,17376-07,Modifica la ley N° 20.084 para autorizar el tr...,None
3,17357-18,Modifica la ley N° 14.908 para perfeccionar lo...,None
4,17356-06,"Modifica la ley N° 19.175, sobre gobierno y ad...",None
